In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import cv2
from google.colab.patches import cv2_imshow
import math
import numpy as np
import mediapipe as mp
import pandas as pd
import os
from tqdm import tqdm

1. Images

In [7]:
!unzip '/content/gdrive/MyDrive/yoga_dataset_images_final.zip' -d 'dataset'


Streaming output truncated to the last 5000 lines.
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_295.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_296.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_298.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_299.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_3.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_30.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_31.jpg  
  inflating: dataset/yoga_dataset_images_final/Supported_Shoulderstand_pose_or_Salamba_Sarvangasana_/2_32.jpg  
  inflating: dataset/yoga_dataset_images_final/Sup

In [8]:
temp = os.listdir("dataset/yoga_dataset_images_final")

In [9]:
dict2 = {}
for i in range(1,83):
  dict2['folder'+str(i)] = temp[i-1]

In [10]:
dict1 = {}
for i in range(1,83):
  folderName = "folder"+str(i)
  dict1[folderName] = os.listdir("/content/dataset/yoga_dataset_images_final/"+str(dict2[folderName]))

In [11]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils 
mp_drawing_styles = mp.solutions.drawing_styles

In [12]:
columnNames = []
for data in mp_pose.PoseLandmark:
  temp_col = data.name
  columnNames.append(temp_col+"-x")
  columnNames.append(temp_col+"-y")
  columnNames.append(temp_col+"-z")
  columnNames.append(temp_col+"-v")
columnNames.append("Pose")
columnNames.append("ImgNum")
trainSet = pd.DataFrame(columns=columnNames)
discardedTrain = pd.DataFrame(columns=["Pose","ImgNum"])

In [14]:
#for each pose, for each image
with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5, model_complexity=2) as pose:
  for foldername in tqdm(dict1.keys()):
    folder = dict1[foldername]
    for imgname in folder:
      if imgname == '.ipynb_checkpoints':
        continue
      img = cv2.imread("/content/dataset/yoga_dataset_images_final/"+dict2[foldername]+"/"+imgname)
      #cv2_imshow(img)

      try:
        storagetemp = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      except:
        discardedTrain.loc[len(discardedTrain.index)] = [dict2[foldername], imgname]
        continue

      results = pose.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
      if (results.pose_landmarks==None):
        discardedTrain.loc[len(discardedTrain.index)] = [dict2[foldername], imgname]
        continue
      #add result to trainSet
      row = []
      for i in range(0,33):
        coord = results.pose_landmarks.landmark[i]
        row.append(coord.x)
        row.append(coord.y)
        row.append(coord.z)
        row.append(coord.visibility)
      row.append(dict2[foldername])
      row.append(imgname)
      trainSet.loc[len(trainSet.index)] = row

100%|██████████| 82/82 [1:24:34<00:00, 61.89s/it]


In [19]:
trainSet.to_csv('/content/gdrive/MyDrive/trainSet.csv', encoding='utf-8', index=False)
discardedTrain.to_csv('/content/gdrive/MyDrive/discardedTrain.csv', encoding='utf-8', index=False)